In [44]:
import pandas as pd

# load the file
pymoli = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
pymoli_df = pd.read_csv(pymoli).dropna()
pymoli_df.head().style.hide_index()

Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.530000
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.560000
2,Ithergue48,24,Male,92,Final Critic,4.880000
3,Chamassasya86,24,Male,100,Blindscythe,3.270000
4,Iskosia90,23,Male,131,Fury,1.440000


In [45]:
# Create player demographics dataframe for later use
demographics = pymoli_df[["Gender", "SN", "Age"]].drop_duplicates()

# Create player_count variable for later use
player_count = pymoli_df["SN"].nunique()

total_players_df = pd.DataFrame({"Total Players": [player_count]}).style.hide_index()
total_players_df

Total Players
576


In [48]:
#check if there are incomplete rows
pymoli_df.count()


Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [60]:
#Purchasing Analysis (Total)
unique_items = pymoli_df["Item ID"].nunique()
average_price = round(pymoli_df["Price"].mean(),2)
number_purchases = pymoli_df["Purchase ID"].nunique()
total_revenue = round(pymoli_df["Price"].sum(),2)


Values = pd.DataFrame({"Number of Unique Items": [unique_items], 
                       "Average Price": [average_price], 
                       "Number of Purchases": [number_purchases], 
                       "Total Revenue": [total_revenue]}).style.hide_index()
Values


Number of Unique Items,Average Price,Number of Purchases,Total Revenue
179,3.050000,780,2379.770000


In [46]:
#Gender Demographics






In [ ]:
# Purchasing Analysis (Gender)



